# 自然语言处理介绍及实践

# 1. 基本概念

<img src='./image/nlp.jpg' />

自然语言处理是计算机科学领域与人工智能领域中的一个重要方向。它研究能实现人与计算机之间用自然语言进行有效通信的各种理论和方法。自然语言处理是一门融语言学、计算机科学、数学于一体的科学。因此，这一领域的研究将涉及自然语言，即人们日常使用的语言，所以它与语言学的研究有着密切的联系，但又有重要的区别。自然语言处理并不是一般地研究自然语言，而在于研制能有效地实现自然语言通信的计算机系统，特别是其中的软件系统。因而它是计算机科学的一部分。

自然语言处理（NLP）是计算机科学，人工智能，语言学关注计算机和人类（自然）语言之间的相互作用的领域。

作为data analyst，我们日常中的工作，很大一部分就是将信息从交易所、上市公司、基金公司公布的金融文档中提取出来。

比如基金名称，具体的林林总总的金融数据等，如果掌握自然语言处理技巧，或许能够对日常工作如虎添翼。

# 2. 主要范畴

文本朗读（Text to speech）/语音合成（Speech synthesis）

语音识别（Speech recognition）

中文自动分词（Chinese word segmentation）

词性标注（Part-of-speech tagging）

句法分析（Parsing）

自然语言生成（Natural language generation）

文本分类（Text categorization）

信息检索（Information retrieval）

信息抽取（Information extraction）

文字校对（Text-proofing）

问答系统（Question answering）

机器翻译（Machine translation）

自动摘要（Automatic summarization）

文字蕴涵（Textual entailment）

<img src='./image/nlparc.jpg' />

# 3. 常用套路

## 3.1 收集数据

对于我们analyst来说，就是从我们文档库里面，把我们关心的filing收集起来，然后最好按照句子为单位作为样本进行堆叠。

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

我们拿如下这一段话，进行分句：

This prospectus offers variable annuity contract allowing you to accumulate values and paying you benefits on a variable and/or fixed basis. This prospectus provides information regarding the material provisions of your variable annuity contract. We may restrict the availability of this contract to certain broker-dealers. National Security Life V.I. and Annuity Company ("National Security") issues the contract. This contract is only available in New York.

In [2]:
def splitparagraph2sentence(paragraph):
    doc = nlp(paragraph)
    return [sentence.text for sentence in doc.sents]

In [3]:
sentences = splitparagraph2sentence('This prospectus offers variable annuity contract allowing you to accumulate values and paying you benefits on a variable and/or fixed basis. This prospectus provides information regarding the material provisions of your variable annuity contract. We may restrict the availability of this contract to certain broker-dealers. National Security Life V.I. and Annuity Company ("National Security") issues the contract. This contract is only available in New York.')
for sentence in sentences:
    print(sentence)

This prospectus offers variable annuity contract allowing you to accumulate values and paying you benefits on a variable and/or fixed basis.
This prospectus provides information regarding the material provisions of your variable annuity contract.
We may restrict the availability of this contract to certain broker-dealers.
National Security Life V.I. and Annuity Company ("National Security") issues the contract.
This contract is only available in New York.


注意：National Security Life V.I.中的点，没有被无脑作为分句的依据，而是真正根据语义分句。

<b>如果能够做有监督的分类，就顺手打上标签</b>，因为做无监督的聚类操作，然后根据相似度判断文本类型，耗时耗力，而且效果不是很好。

## 3.2 清洗数据

我们遵循的第一原则是：“再好的模型也拯救不了shi一样的数据”。所以，先来清洗一下数据吧！

我们做以下处理：
准则：去除变量，只留常量，或者可常量化。

1. 删除所有不相关的字符，如任何非字母数字字符

2. 通过文本分隔分成单独的单词来标记你的文章

3. 删除不相关的字词，例如“@”推特或网址

4. 将所有字符转换为小写字母，以便将诸如“hello”，“Hello”和“HELLO”等单词看做相同单词

5. 考虑整合拼写错误或多种拼写的单词，用一个单词代表（例如“cool”/“kewl”/“cooool”）相结合

6. 考虑词形还原（把“am”，“are”，“is”等词语缩小为“be”这样的常见形式）

7. 将所有专有名词转换为propn这个语义标注词，即变量转换为常量！

8. 去除停用词，比如for a an of the and to about after in among as...

具体实现方式：

### 删除所有非字母的字符
如这句话：Also assume that, when the Owner is age 76, a step up occurs and the highest quarterly Contract Value is greater than the BDB; in that case, the GAWA percentage will be re determined based on the Owner's attained age of 76, resulting in a new GAWA percentage of 6%.

In [4]:
import re
text = '''Also assume that, when the Owner is age 76, a step up occurs and the highest quarterly Contract Value is greater than the BDB; in that case, the GAWA percentage will be re determined based on the Owner's attained age of 76, resulting in a new GAWA percentage of 6%.'''
text = re.sub(r'\W', ' ', text)
text = re.sub(r'\d', ' ', text)
text = re.sub(r'( ){2,}', ' ', text).strip()
print(text)

Also assume that when the Owner is age a step up occurs and the highest quarterly Contract Value is greater than the BDB in that case the GAWA percentage will be re determined based on the Owner s attained age of resulting in a new GAWA percentage of


### 词性还原

#### 什么是词性？

词性指以词的特点作为划分词类的根据，比如：
ADV: 副词；sample：very, well, exactly, tomorrow, up, down

VERB: 动词；sample: run, eat, ate, running, eats

ADJ: 形容词；sample: big, old, green

DET: 限定词；sample: a, an, this, this, no

NOUN: 名词；sample: girl, boy, cat, tree

ADP: 介词；sample: in, to, during

PROPN: 专属名词；sample: Mary, London, HBO, Google

CCONJ: 连词；sample: and, or, but

参照：http://universaldependencies.org/u/pos/all.html

下面的例子，演示如何通过Spacy获取一句话中各个单词的词性

In [5]:
def getwordtokenattributes(text):
    doc = nlp(text)
    result = []
    wordlist = []
    for token in doc:
#         if token.text not in wordlist:
        dictinfo = {}
        dictinfo['text'] = token.text
        dictinfo['lemma_'] = token.lemma_
        dictinfo['pos_'] = token.pos_
        dictinfo['tag_'] = token.tag_
        dictinfo['dep_'] = token.dep_
        dictinfo['shape_'] = token.shape_
        dictinfo['is_alpha'] = token.is_alpha
        dictinfo['is_stop'] = token.is_stop
        wordlist.append(token.text)
        result.append(dictinfo)
    return result

In [6]:
result = getwordtokenattributes(r"It's supposed to be removed.")

In [7]:
print(result)

[{'text': 'It', 'lemma_': '-PRON-', 'pos_': 'PRON', 'tag_': 'PRP', 'dep_': 'nsubjpass', 'shape_': 'Xx', 'is_alpha': True, 'is_stop': False}, {'text': "'s", 'lemma_': 'be', 'pos_': 'VERB', 'tag_': 'VBZ', 'dep_': 'auxpass', 'shape_': "'x", 'is_alpha': False, 'is_stop': False}, {'text': 'supposed', 'lemma_': 'suppose', 'pos_': 'VERB', 'tag_': 'VBN', 'dep_': 'ROOT', 'shape_': 'xxxx', 'is_alpha': True, 'is_stop': False}, {'text': 'to', 'lemma_': 'to', 'pos_': 'PART', 'tag_': 'TO', 'dep_': 'aux', 'shape_': 'xx', 'is_alpha': True, 'is_stop': True}, {'text': 'be', 'lemma_': 'be', 'pos_': 'VERB', 'tag_': 'VB', 'dep_': 'auxpass', 'shape_': 'xx', 'is_alpha': True, 'is_stop': True}, {'text': 'removed', 'lemma_': 'remove', 'pos_': 'VERB', 'tag_': 'VBN', 'dep_': 'xcomp', 'shape_': 'xxxx', 'is_alpha': True, 'is_stop': False}, {'text': '.', 'lemma_': '.', 'pos_': 'PUNCT', 'tag_': '.', 'dep_': 'punct', 'shape_': '.', 'is_alpha': False, 'is_stop': False}]


我们可以用Pandas的DataFrame，将结果变得容易阅读：

In [8]:
import pandas as pd
df = pd.DataFrame(result)
df

,dep_,is_alpha,is_stop,lemma_,pos_,shape_,tag_,text
0,nsubjpass,True,False,-PRON-,PRON,Xx,PRP,It
1,auxpass,False,False,be,VERB,'x,VBZ,'s
2,ROOT,True,False,suppose,VERB,xxxx,VBN,supposed
3,aux,True,True,to,PART,xx,TO,to
4,auxpass,True,True,be,VERB,xx,VB,be
5,xcomp,True,False,remove,VERB,xxxx,VBN,removed
6,punct,False,False,.,PUNCT,.,.,.


#### 通过词性还原获得语干

In [9]:
def lemmatization(sentence, allowed_postags=''):
    """https://spacy.io/api/annotation"""
    doc = nlp(sentence)
    # allowed_postags, such as 'NOUN,ADJ,VERB,ADV',
    # 但是大多数情况，不能加allow_postags，否则很多词，比如no,  or就没有了
    if len(allowed_postags) > 0:
        resultlist = [token.lemma_
                      for token
                      in doc
                      if token.pos_
                      in [postag.upper().strip() for postag in allowed_postags.split(',')]]
    else:
        resultlist =  [token.lemma_ for token in doc]
    return resultlist

In [10]:
text = r'The product is the best than others.'

In [11]:
print(' '.join(lemmatization(text)))

the product be the good than other .


#### 通过词性表达式获得短语

In [12]:
import textacy
def extractverbphrase(text, pattern=r'(<ADV>*<NOUN|PROPN>*<VERB><DET>?<ADV>*<VERB|ADJ>+<ADP>?<DET>?<NUM>*<ADJ>*<NOUN|PROPN>*<ADV>?)|(<VERB>?<NOUN|PROPN>*<ADV>?<VERB><ADP>?<ADJ|VERB>*<ADP>?<DET>?<VERB>?<NOUN|PROPN>*)|(<DET>?<ADJ>+<NOUN|PROPN>+)|(<ADV>*<ADJ><ADP><DET>?<VERB|ADJ>*<NOUN|PROPN>*)|(<DET><NOUN><CCONJ><NOUN>)|(<NOUN|PROPN>*<CCONJ>?<NOUN|PROPN>+<ADP><NOUN|PROPN>+)|(<ADP><DET><NOUN|PROPN>+)'):
    # ADV: 副词；sample：very, well, exactly, tomorrow, up, down
    # VERB: 动词；sample: run, eat, ate, running, eats
    # ADJ: 形容词；sample: big, old, green
    # DET: 限定词；sample: a, an, this, this, no
    # NOUN: 名词；sample: girl, boy, cat, tree
    # ADP: 介词；sample: in, to, during
    # PROPN: 专属名词；sample: Mary, London, HBO, Google
    # CCONJ: 连词；sample: and, or, but
    # 参照：http://universaldependencies.org/u/pos/all.html
    doc = nlp(text)
    return list(textacy.extract.pos_regex_matches(doc, pattern))

In [13]:
text = r'Effective April 24, 2017, there are new Investment Divisions for which Accumulation Unit information is not yet available.'
phraselist = extractverbphrase(text, pattern=r'(<PROPN>+)')
for phrase in phraselist:
    print(phrase)

April
Investment Divisions
Accumulation Unit


#### 统一的文字清洗方法

将清理逻辑连接起来，构成一个统一的文字清洗方法：

In [14]:
import re

In [15]:
def removespecialchar(sentence):
    result = re.sub('\W', ' ', sentence)
    return re.sub('( ){2,}', ' ', result)

In [16]:
def clearandlemmasentence(sentence,
                          stopword='for a an the and in among'):
    stoplist = set(stopword.split())
    sentence = removespecialchar(sentence).lower().strip()
    sentence = ' '.join([word.strip() for word
                         in sentence.lower().strip().split()
                         if len(word.strip()) > 0
                         and word not in stoplist]).strip()
    sentence = re.sub(r'(propn\s+){2,}', 'propn ', sentence)
    if len(sentence) == 0:
        sentence = 'only for test'
    lemmawordlist = lemmatization(sentence)
    return lemmawordlist

In [17]:
def replacevariabletextfromtextblock(textblock):
    """
    Variable Text:
    1. PROPN words, such as: Mainstay VP Funds Trust, replace them with propn
    2. Date part, such as January 1, 2018, replace them with date
    3. Number, such as 1, 2, replace with space
    :param textblock:
    :return:
    """
    # replace date string with "date"
    datepattern = r'((January|February|March|April|May|June|July|August|September|October|November|December)[\s]*[0-9]{1,2}[\s]*,[\s]*[0-9]{4})|([0-9]{1,2}/[0-9]{1,2}/[0-9]{4})'
    textblock = re.sub(datepattern, 'date', textblock)
    datepattern = r'\d{2}\/\d{2}\/(\d{4}|\d{2})'
    textblock = re.sub(datepattern, 'date', textblock)
    # 应对*CTIVP这种情况，无法识别PROPN
    textblock = textblock.replace('*', ' ')
    textblock = re.sub(r'( ){2,}', ' ', textblock).strip()
    # 因为Money Market Fund前缀与后缀词经常是具体的基金公司，
    # 所以去除具体基金公司名称的同时，
    # 避免其被作为专属名词替换
    textblock = textblock.replace(' of ', ' ')\
        .replace(' Inc.', ' ')\
        .replace('&', '')\
        .replace(' LLC ', ' ')\
        .replace(' BlackRock ', ' ')\
        .replace(' SP ', ' ')
    textblock = textblock.replace('-', ' ').\
        replace('–', ' ').\
        replace('Addition', 'addition')
    textblock = re.sub(r'\d', ' ', textblock)
    textblock = re.sub(r'( ){2,}', ' ', textblock).strip()
    phraselist = extractverbphrase(textblock, '<PROPN>+')
    phraselist.sort(key=lambda i: len(i), reverse=True)
    if len(phraselist) > 0:
        for phrase in phraselist:
            phrasetext = phrase.text
            # avoid remove important words which are related with category
            if 'money market fund' in phrasetext.lower():
                textblock = textblock.replace(phrasetext, 'money market fund')
            noexcludewordlist = ['date',
                                 ' merge ',
                                 ' merged ',
                                 ' merging ',
                                 ' merger ',
                                 'acquir',
                                 'survive',
                                 'surviving',
                                 'survived',
                                 'liquidat',
                                 'transfer',
                                 'reorganiz',
                                 'expense table',
                                 'fee summary',
                                 'operating expenses',
                                 'annual fund',
                                 'the adviser',
                                 'benefit payment',
                                 'variable account option',
                                 ' new ']
            shouldignore = False
            for word in noexcludewordlist:
                if word in phrasetext.lower():
                    shouldignore = True
                    break
            if shouldignore:
                continue
            if not any([phrasetext.lower() == 'fund',
                        len(phrasetext.split()) <= 2]):
                textblock = textblock.replace(phrasetext, 'propn')
    textblock = textblock.replace('PIMCO', ' ')
    textblock = re.sub(r'\W', ' ', textblock)
    textblock = re.sub(r'(propn\s+){2,}', 'propn ', textblock)
    textblock = re.sub(r'( ){2,}', ' ', textblock).strip()
    return textblock

In [18]:
def cleardatafordoc2vector(doc):
    temp = ' '.join(
        clearandlemmasentence(replacevariabletextfromtextblock(doc),
                              'for a an of the and or to about after in among as at be been was were is are being b c d e f g h i j k l m n o p q r s t u v w x y z'
                              )).strip()
    temp = temp.replace('-PRON-', 'pron')
    return temp

现在可以做一下效果测试：<br>
原句：122 66 32 15 14 5 13 17 *CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018. 

In [19]:
text = r'122 66 32 15 14 5 13 17 *CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018. '
print(cleardatafordoc2vector(text))

propn class liquidate on date


## 3.3 找到一个好的数据表示方式

### 3.3.1 词袋化

Bag-of-words模型是信息检索领域常用的文档表示方法。

在信息检索中，BOW模型假定对于一个文档，忽略它的单词顺序和语法、句法等要素，将其仅仅看作是若干个词汇的集合，文档中每个单词的出现都是独立的，不依赖于其它单词是否出现。

也就是说，文档中任意一个位置出现的任何单词，都不受该文档语意影响而独立选择的。

词袋模型的缺点：

词袋模型最重要的是构造词表，然后通过文本为词表中的词赋值，但词袋模型严重缺乏相似词之间的表达。 

比如“我喜欢北京”“我不喜欢北京”其实这两个文本是严重不相似的。但词袋模型会判为高度相似。 

“我喜欢北京”与“我爱北京”其实表达的意思是非常非常的接近的，但词袋模型不能表示“喜欢”和“爱”之间严重的相似关系。（当然词袋模型也能给这两句话很高的相似度，但是注意我想表达的含义）

在Investment名字相似度这个应用中，正是采用了词袋 + TF/IDF模型 + 余弦相似度作为核心。因为单纯的investment并不存在或者很少存在需要语义分析。

下面是代码示例：

In [20]:
from gensim import corpora, models, similarities

d:\python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
namelist = [
    'ODDO BHF US Mid Cap CI-EUR H',
    'ODDO BHF US Mid Cap CR-USD',
    'Credit Suisse Index Fund (CH) - CSIF (CH) Bond Fiscal Strength EUR Blue ZA',
    'Winton Diversified Futures Fund (Luxembourg) C GBP Acc',
    'Prescient Core Equity Fund B5',
    'Robeco QI GTAA Plus DHL $',
    'Franklin US Rising Dividends T',
    'FT MLP Closed-End Fund & Energy 52 CA',
    'FT Richard Bern Adv TS Amer Ind 16-3 CA',
    'FT Municipal FT Income Select CE 81 CA',
    'Raiffeisen-Pensionsfonds-Österreich 2007 VT',
    'Multipartner SICAV - Carthesio Asian Credit Fund B EUR',
    'HSBC Wealth Strategic Solutions Fund (1) - Conservative Portfolio Income X',
    'American Beacon Flexible Bond Fund A Class',
    'Robeco QI GTAA Plus IHL $',
    'AXA World Funds - Global Equity Income M Capitalisation EUR']
stoplist = set('for a an of the and to in - $ &'.split())

In [22]:
data_train = []
for name in namelist:
    data_train.append([word for word in name.strip().split() 
                       if word not in stoplist])
print(data_train)

[['ODDO', 'BHF', 'US', 'Mid', 'Cap', 'CI-EUR', 'H'], ['ODDO', 'BHF', 'US', 'Mid', 'Cap', 'CR-USD'], ['Credit', 'Suisse', 'Index', 'Fund', '(CH)', 'CSIF', '(CH)', 'Bond', 'Fiscal', 'Strength', 'EUR', 'Blue', 'ZA'], ['Winton', 'Diversified', 'Futures', 'Fund', '(Luxembourg)', 'C', 'GBP', 'Acc'], ['Prescient', 'Core', 'Equity', 'Fund', 'B5'], ['Robeco', 'QI', 'GTAA', 'Plus', 'DHL'], ['Franklin', 'US', 'Rising', 'Dividends', 'T'], ['FT', 'MLP', 'Closed-End', 'Fund', 'Energy', '52', 'CA'], ['FT', 'Richard', 'Bern', 'Adv', 'TS', 'Amer', 'Ind', '16-3', 'CA'], ['FT', 'Municipal', 'FT', 'Income', 'Select', 'CE', '81', 'CA'], ['Raiffeisen-Pensionsfonds-Österreich', '2007', 'VT'], ['Multipartner', 'SICAV', 'Carthesio', 'Asian', 'Credit', 'Fund', 'B', 'EUR'], ['HSBC', 'Wealth', 'Strategic', 'Solutions', 'Fund', '(1)', 'Conservative', 'Portfolio', 'Income', 'X'], ['American', 'Beacon', 'Flexible', 'Bond', 'Fund', 'A', 'Class'], ['Robeco', 'QI', 'GTAA', 'Plus', 'IHL'], ['AXA', 'World', 'Funds', 'Glo

下面的代码演示如何生成词袋字典以及词袋模型，并保存为具体的文件

In [23]:
dictionary = corpora.Dictionary(data_train)
print('输出每个单词对应的索引编号')
print(dictionary.token2id)
dictpath = './nlpmodel/corpus.dict'
dictionary.save(dictpath)
corpus = [dictionary.doc2bow(text) for text in data_train]
print('输出当前句子中各个单词的索引编号以及出现频率')
for corpu in corpus:
    print(corpu)
modelpath = './nlpmodel/corpus.mm'
corpora.MmCorpus.serialize(modelpath, corpus)

输出每个单词对应的索引编号
{'BHF': 0, 'CI-EUR': 1, 'Cap': 2, 'H': 3, 'Mid': 4, 'ODDO': 5, 'US': 6, 'CR-USD': 7, '(CH)': 8, 'Blue': 9, 'Bond': 10, 'CSIF': 11, 'Credit': 12, 'EUR': 13, 'Fiscal': 14, 'Fund': 15, 'Index': 16, 'Strength': 17, 'Suisse': 18, 'ZA': 19, '(Luxembourg)': 20, 'Acc': 21, 'C': 22, 'Diversified': 23, 'Futures': 24, 'GBP': 25, 'Winton': 26, 'B5': 27, 'Core': 28, 'Equity': 29, 'Prescient': 30, 'DHL': 31, 'GTAA': 32, 'Plus': 33, 'QI': 34, 'Robeco': 35, 'Dividends': 36, 'Franklin': 37, 'Rising': 38, 'T': 39, '52': 40, 'CA': 41, 'Closed-End': 42, 'Energy': 43, 'FT': 44, 'MLP': 45, '16-3': 46, 'Adv': 47, 'Amer': 48, 'Bern': 49, 'Ind': 50, 'Richard': 51, 'TS': 52, '81': 53, 'CE': 54, 'Income': 55, 'Municipal': 56, 'Select': 57, '2007': 58, 'Raiffeisen-Pensionsfonds-Österreich': 59, 'VT': 60, 'Asian': 61, 'B': 62, 'Carthesio': 63, 'Multipartner': 64, 'SICAV': 65, '(1)': 66, 'Conservative': 67, 'HSBC': 68, 'Portfolio': 69, 'Solutions': 70, 'Strategic': 71, 'Wealth': 72, 'X': 73, 'A': 74, 

下文将演示如何通过TF/IDF模型求语句相似度：

In [24]:
# 初始化模型
corpus = corpora.MmCorpus(modelpath)
dictionary = corpora.Dictionary.load(dictpath)
tfidf_model = models.TfidfModel(corpus)
index = similarities.SparseMatrixSimilarity(
    tfidf_model[corpus],
    num_features=len(dictionary.keys()))

In [25]:
print('准备测试语句')
testtext = 'CR-USD ODDO Mid Cap Cap BHF US'.split()
doc_text_vec = dictionary.doc2bow(testtext)
print(doc_text_vec)

准备测试语句
[(0, 1), (2, 2), (4, 1), (5, 1), (6, 1), (7, 1)]


In [26]:
print('直接通过TF/IDF模型获取相似度, 返回数值越大，相似度越高')
print(index.get_similarities(doc_text_vec))

直接通过TF/IDF模型获取相似度, 返回数值越大，相似度越高
[2.0267534  2.8160036  0.         0.         0.         0.
 0.28899837 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [27]:
test_simi = index[tfidf_model[doc_text_vec]]
test_simi = sorted(enumerate(test_simi), key=lambda item: -item[1])
outputlist = [test for test in test_simi if test[1] > 0.2]
print(outputlist)

[(1, 0.9541586), (0, 0.6422975)]


如果想看与哪一句最相似，直接使用索引，从语料包拿就可以

In [28]:
print('raw sentence: ', ' '.join(testtext))
for output in outputlist:
    print(namelist[output[0]],'---------similarity: ', output[1])

raw sentence:  CR-USD ODDO Mid Cap Cap BHF US
ODDO BHF US Mid Cap CR-USD ---------similarity:  0.9541586
ODDO BHF US Mid Cap CI-EUR H ---------similarity:  0.6422975


### 3.3.2 Doc2Vector中的TaggedDocument

Doc2Vector其实与Word2Vector类似，都有语义分析成分，但是索引单位是句子

Doc2Vector的训练集的组成单元是TaggedDocument对象, 如下是官方说明：

Represents a document along with a tag, input document format for class: `gensim.models.doc2vec.Doc2Vec`.

A single document, made up of `words` (a list of unicode string tokens) and `tags` (a list of tokens).

Tags may be one or more unicode string tokens, but typical practice (which will also be the most memory-efficient) is for the tags list to include a unique integer id as the only tag.

In [29]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

In [30]:
sentencelist = [
    'may prize winner teacher bomb',
    'production value use cgi digital ink paint make thing look really slick voice fine well problem thing script',
    'got heart right place also wilt awhile',
    'prof movie goodness thing good movie',
    'well go forever',
    'overproduced generally disappointing effort likely rouse rush hour crowd']
x_train = []
for index, sentence in enumerate(sentencelist):
    document = TaggedDocument(sentence.split(), tags=['{0}'.format(index)])
    print(document)
    x_train.append(document)
# model_dm = Doc2Vec(x_train, min_count=1, window=3, size=200, sample=1e-3, negative=5, workers=2)
# print(model_dm)

TaggedDocument(['may', 'prize', 'winner', 'teacher', 'bomb'], ['0'])
TaggedDocument(['production', 'value', 'use', 'cgi', 'digital', 'ink', 'paint', 'make', 'thing', 'look', 'really', 'slick', 'voice', 'fine', 'well', 'problem', 'thing', 'script'], ['1'])
TaggedDocument(['got', 'heart', 'right', 'place', 'also', 'wilt', 'awhile'], ['2'])
TaggedDocument(['prof', 'movie', 'goodness', 'thing', 'good', 'movie'], ['3'])
TaggedDocument(['well', 'go', 'forever'], ['4'])
TaggedDocument(['overproduced', 'generally', 'disappointing', 'effort', 'likely', 'rouse', 'rush', 'hour', 'crowd'], ['5'])


### 3.3.3 Keras中的Tokenizer与pad_sequences

#### text.Tokenizer类

这个类用来对文本中的词进行统计计数，生成文档词典，以支持基于词典位序生成文本的向量表示。 
init(num_words) 构造函数，传入词典的最大值

##### 成员函数

- fit_on_text(texts) 使用一系列文档来生成token词典，texts为list类，每个元素为一个文档。
- texts_to_sequences(texts) 将多个文档转换为word下标的向量形式,shape为`[len(texts)，len(text)]` -- (文档数，每条文档的长度)
- texts_to_matrix(texts) 将多个文档转换为矩阵表示,shape为`[len(texts),num_words]`

##### 成员变量

- document_count 处理的文档数量
- word_index 一个dict，保存所有word对应的编号id，从<b>1</b>开始
- word_counts 一个dict，保存每个word在所有文档中出现的次数
- word_docs 一个dict，保存每个word出现的文档的数量
- index_docs 一个dict，保存word的id出现的文档的数量

示例：

In [31]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.


In [32]:
sentencelist = [
    'may prize winner teacher bomb',
    'production value use cgi digital ink paint make thing look really slick voice fine well problem thing script',
    'got heart right place also wilt awhile',
    'prof movie goodness thing good movie',
    'well go forever',
    'overproduced generally disappointing effort likely rouse rush hour crowd']

In [33]:
print('text_to_word_sequence的用法与字符串的split用法类似')
print(text_to_word_sequence(sentencelist[0]))

text_to_word_sequence的用法与字符串的split用法类似
['may', 'prize', 'winner', 'teacher', 'bomb']


In [34]:
max_fatures = 2000

In [35]:
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(sentencelist)
print('tokenizer.word_counts')
print(tokenizer.word_counts)
print()
print('tokenizer.word_index')
print(tokenizer.word_index)
print()
print('tokenizer.word_docs')
print(tokenizer.word_docs)

print()
print('tokenizer.index_docs')
print(tokenizer.index_docs) 

tokenizer.word_counts
OrderedDict([('may', 1), ('prize', 1), ('winner', 1), ('teacher', 1), ('bomb', 1), ('production', 1), ('value', 1), ('use', 1), ('cgi', 1), ('digital', 1), ('ink', 1), ('paint', 1), ('make', 1), ('thing', 3), ('look', 1), ('really', 1), ('slick', 1), ('voice', 1), ('fine', 1), ('well', 2), ('problem', 1), ('script', 1), ('got', 1), ('heart', 1), ('right', 1), ('place', 1), ('also', 1), ('wilt', 1), ('awhile', 1), ('prof', 1), ('movie', 2), ('goodness', 1), ('good', 1), ('go', 1), ('forever', 1), ('overproduced', 1), ('generally', 1), ('disappointing', 1), ('effort', 1), ('likely', 1), ('rouse', 1), ('rush', 1), ('hour', 1), ('crowd', 1)])

tokenizer.word_index
{'thing': 1, 'well': 2, 'movie': 3, 'may': 4, 'prize': 5, 'winner': 6, 'teacher': 7, 'bomb': 8, 'production': 9, 'value': 10, 'use': 11, 'cgi': 12, 'digital': 13, 'ink': 14, 'paint': 15, 'make': 16, 'look': 17, 'really': 18, 'slick': 19, 'voice': 20, 'fine': 21, 'problem': 22, 'script': 23, 'got': 24, 'heart

In [36]:
sequences = tokenizer.texts_to_sequences(sentencelist)
print(sequences)

[[4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16, 1, 17, 18, 19, 20, 21, 2, 22, 1, 23], [24, 25, 26, 27, 28, 29, 30], [31, 3, 32, 1, 33, 3], [2, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44]]


One_Hot化

In [37]:
print(tokenizer.texts_to_matrix(sentencelist))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


pad_sequences非常重要，目的是将序列填充到maxlen长度，不足maxlenth的句子，用0填充

<b><font color='red'>这个非常重要，Keras用于做分类训练的样本，需要通过填充对齐，才能进行之后的训练</font></b>

In [38]:
X = pad_sequences(sequences, maxlen=20)
print(X)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4  5  6  7  8]
 [ 0  0  9 10 11 12 13 14 15 16  1 17 18 19 20 21  2 22  1 23]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 24 25 26 27 28 29 30]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 31  3 32  1 33  3]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2 34 35]
 [ 0  0  0  0  0  0  0  0  0  0  0 36 37 38 39 40 41 42 43 44]]


## 3.4 训练模型

文本清洗，词袋化或“向量化”（这里向量化打引号，表示与真正的词向量概率不同，这里仅仅是将词或句建立向量索引）之后，就是建模了。

上述部分已经提及了如何创建TF/IDF这种简单模型，那么如何创建词向量模型(word2vec)，句向量(doc2vec)以及通过Keras创建LSTM, biLSTM, GRU乃至biGRU模型呢？

我们将在`5. 常用模型`这一章详细了解。

## 3.5 使用模型

我们将在`5. 常用模型`这一章详细了解。

# 4. 常用自然语言处理包

工欲善其事，必先利其器。目前为止已经有很多很多用于NLP专项应用的python包。

下面将逐一介绍它们。

## 4.1 Gensim

<img src='./image/gensimoffice.png' />

Gensim是一个用于从文档中自动提取语义主题的Python库，足够智能。

Gensim可以处理原生，非结构化的数值化文本(纯文本)。Gensim里面的算法，比如Latent Semantic Analysis(潜在语义分析LSA)，Latent Dirichlet Allocation，Random Projections，通过在语料库的训练下检验词的统计共生模式(statistical co-occurrence patterns)来发现文档的语义结构。

这些算法是无监督的，也就是说你只需要一个语料库的文档集。

当得到这些统计模式后，任何文本都能够用语义表示(semantic representation)来简洁的表达，并得到一个局部的相似度与其他文本区分开来。

常用的功能有：语料(Corpus)，TF/IDF、LSA、LDA模型，Word2Vec, Doc2Vec, 以及通过各种模型获得单词、语句间的相似度。这些都是无监督使用方式。

但是，如果Doc2Vec的TaggedDocument使用得当，甚至可以起到有监督的语句分类的功效。

安装方式：`pip install -U gensim`

可以从Github获取源码：[Gensim on Github](https://github.com/RaRe-Technologies/gensim)

<img src='./image/gensimongithub.png' />

官方站点：[Gensim](https://radimrehurek.com/gensim/)

<img src='./image/gensimoffice2.png' />

第三方文档教程：[gensim](https://kite.com/python/docs/gensim)

<img src='./image/gensimonkite.png' />

很有特色的英文教程：[Gensim Tutorial – A Complete Beginners Guide](https://www.machinelearningplus.com/nlp/gensim-tutorial/)

<img src='./image/gensimtutorial.png' />

## 4.2 Spacy

Spacy是由`Cython`编写，因此它是一个非常快的库，可以说是工业级别的NLP组件库。

Spacy源自预训练统计模型，词向量，并且支持30+语言。

其号称具有最快的语法解析器，通过卷积神经网络CNN（convolutional neural network models）做token标注、解析以及命名实体识别，并且很方便做深度学习整合应用。

Spacy是商业开源软件，基于MIT license发布。

Spacy的特点与功能：

- Fastest syntactic parser in the world
- Named entity recognition
- Non-destructive tokenization
- Support for 30+ languages
- Pre-trained statistical models and word vectors
- Easy deep learning integration
- Part-of-speech tagging
- Labelled dependency parsing
- Syntax-driven sentence segmentation
- Built in visualizers for syntax and NER
- Convenient string-to-hash mapping
- Export to numpy data arrays
- Efficient binary serialization
- Easy model packaging and deployment
- State-of-the-art speed
- Robust, rigorously evaluated accuracy

安装方式：`pip install spacy`或者`pip install -U spacy`

在第三章中，我们使用Spacy做了分句，单词原型化，根据词性表达式获取短语等应用，而这一切都是是基于模型应用的，下面介绍如何下载语言模型。

最简单的下载模型方式，是基于Spacy的download命令：

```
# out-of-the-box: download best-matching default model
python -m spacy download en
python -m spacy download de
python -m spacy download es
python -m spacy download pt
python -m spacy download fr
python -m spacy download it
python -m spacy download nl
python -m spacy download xx

# download best-matching version of specific model for your spaCy installation
python -m spacy download en_core_web_sm

# download exact model version (doesn't create shortcut link)
python -m spacy download en_core_web_sm-2.0.0 --direct
```

模型下载相关的文档位于：[Models Overview](https://spacy.io/models/)

如果想获得特别全的英文词向量模型，可以下载：`python -m spacy download en_core_web_lg`

昨天举了很多Spacy有关的例子，这里再举一个命名实体识别的例子

In [39]:
import spacy
nlp = spacy.load('en')
doc = nlp("""Mary has a dog, she works for Google, 
and likes to buy things on Amazon, 
she is living in New York.""")
result = {}
for ent in doc.ents:
    if ent.label_ not in result.keys():
        result[ent.label_] = []
    if ent.text.strip() != "":
        result[ent.label_].append(ent.text.strip())
#remove duplicate entity values
for key in result.keys():
    l = []
    for label in result[key]:
        if not label in l:
            l.append(label)
    result[key] = l
print(result)

{'PERSON': ['Mary'], 'ORG': ['Google,', 'Amazon'], 'GPE': ['New York']}


命名实体识别可视化的方式：

通过如下代码，即可通过访问http://localhost:5000 的网址浏览实体识别的具体信息。

In [40]:
from spacy import displacy
displacy.serve(doc, style='ent')


    Serving on port 5000...
    Using the 'ent' visualizer


    Shutting down server on port 5000.



效果大致如下：

<img src='./image/spacyentity.png' />

Github地址：[spaCy: Industrial-strength NLP](https://github.com/explosion/spaCy)

<img src='./image/spacygithub.png' />

官方介绍及教程：[Spacy: Industrial-Strength
Natural Language
Processing](https://spacy.io/)

## 4.3 textacy

textacy是基于Spacy开发的自然语言任务工具，相关特性如下：

- Provide a convenient entry point and interface to one or many documents, with the core processing delegated to spaCy
- Stream text, json, csv, spaCy binary, and other data to and from disk
- Download and explore a variety of included datasets with both text content and metadata, from Congressional speeches to historical literature to Reddit comments
- Clean and normalize raw text, before analyzing it
- Access and filter basic linguistic elements, such as words, ngrams, and noun chunks; extract named entities, acronyms and their definitions, and key terms
- Flexibly tokenize and vectorize documents and corpora, then train, interpret, and visualize topic models using LSA, LDA, or NMF methods
- Compare strings, sets, and documents by a variety of similarity metrics
- Calculate common text statistics, including Flesch-Kincaid Grade Level, SMOG Index, and multilingual Flesch Reading Ease

安装方法：`pip install textacy`

之所以用textacy，是因为其可以预处理文本，比如:
去除URLs: 统一替换为url
Email：统一替换为email
Number: 统一替换为number
标点符号，
重音符号，
HTML标记等，如： 

In [41]:
import textacy
rawtext = """Please visit http://www.google.com, 
then you will get what you want to search. 
There are over 1000 web pages to review.
Please contact me by a@gmail.com."""
text = textacy.preprocess_text(rawtext, 
                               no_urls=True, 
                               no_numbers=True, 
                               no_emails=True,
                               lowercase=True, 
                               no_punct=True)
print(text)

please visit url then you will get what you want to search there are over number web pages to review please contact me by email


In [42]:
help(textacy.preprocess_text)

Help on function preprocess_text in module textacy.preprocess:

preprocess_text(text, fix_unicode=False, lowercase=False, transliterate=False, no_urls=False, no_emails=False, no_phone_numbers=False, no_numbers=False, no_currency_symbols=False, no_punct=False, no_contractions=False, no_accents=False)
    Normalize various aspects of a raw text doc before parsing it with Spacy.
    A convenience function for applying all other preprocessing functions in one go.
    
    Args:
        text (str): raw text to preprocess
        fix_unicode (bool): if True, fix "broken" unicode such as
            mojibake and garbled HTML entities
        lowercase (bool): if True, all text is lower-cased
        transliterate (bool): if True, convert non-ascii characters
            into their closest ascii equivalents
        no_urls (bool): if True, replace all URL strings with '*URL*'
        no_emails (bool): if True, replace all email strings with '*EMAIL*'
        no_phone_numbers (bool): if True, r

除此之外，textacy通过`textacy.Doc`的方式，能够<b>自动检测加载文本的语言</b>。

In [43]:
text = 'Tom is happily running in the park'
doc = textacy.Doc(text)

In [44]:
# help(textacy.Doc)

此外，还提供词性正则表达式的功能，可以方便获取想要的短语组合：

In [45]:
pattern = r'(<NOUN|PROPN>+<VERB>+<DET>?<ADV>*<VERB>+)'
phraselist = list(textacy.extract.pos_regex_matches(doc, pattern))
print([phrase.text for phrase in phraselist])

['Tom is happily running']


textacy的github地址：[textacy: NLP, before and after spaCy](https://github.com/chartbeat-labs/textacy)

<img src='./image/textacygithub.png' />

官方说明文档: [textacy: NLP, before and after spaCy](https://chartbeat-labs.github.io/textacy/index.html)

<img src='./image/textacytutorial.png' />

## 4.4 NLTK

NLTK是一个高效的Python构建的平台，用来处理人类自然语言数据。它提供了易于使用的接口，通过这些接口可以访问超过50个语料库和词汇资源（如WordNet），还有一套用于分类、标记化、词干标记、解析和语义推理的文本处理库，以及工业级NLP库的封装器和一个活跃的讨论论坛。

统计语言学话题方面的手动编程指南加上全面的API文档，使得NLTK非常适用于语言学家、工程师、学生、教育家、研究人员以及行业用户等人群。NLTK可以在Windows、Mac OS X以及Linux系统上使用。最好的一点是，NLTK是一个免费、开源的社区驱动的项目。

NLTK被称为“一个使用Python开发的用于统计语言学的教学和研究的有利工具”和“一个自然语言处理的高效库”。

相比Spacy之类的自然语言处理包，NLTK有一些偏学术化。

Github地址：[NLTK](https://github.com/nltk/nltk)

官方地址：[NLTK](https://www.nltk.org/)

## 4.5 JIEBA

Jieba（结巴）是一个强大的分词库，完美支持中文分词。

一般来说中文文本不会如同拉丁语系一样，词与词之间有明显的空格作为间隔。

如果需要对中文文本建模，那么分词是必须的前提条件，那么Jieba就是目前为止最好的中文分词组件包。

其优点有：

1 支持三种分词模式：

a. 精确模式，试图将句子最精确地切开，适合文本分析；

b. 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；

c. 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

2 支持自定义词典

下面来看看如何对中文做分词：

### 精准模式

试图将句子最精确地切开,适合文本分析

In [46]:
import jieba
text = r'小张毕业于深圳大学，这座大学位于南山区'

In [47]:
words = jieba.cut(text)
print('/'.join(words))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\bhe\AppData\Local\Temp\jieba.cache
Loading model cost 1.194 seconds.
Prefix dict has been built succesfully.


小张/毕业/于/深圳大学/，/这座/大学/位于/南山区


### 全模式

把句子中所有的可以成词的词语都扫描出来, 速度非常快,但是不能解决歧义

In [48]:
words = jieba.cut(text, cut_all=True)
print('/'.join(words))

小张/毕业/于/深圳/深圳大学/大学///这/座/大学/学位/位于/南山/南山区/山区


### 搜索引擎模式

在精确模式的基础上,对长词再次切分,提高召回率,适合用于搜索引擎分词

In [49]:
output = jieba.cut_for_search(text)
print('/'.join(output))

小张/毕业/于/深圳/大学/深圳大学/，/这座/大学/位于/南山/山区/南山区


### 词性标注

In [50]:
import jieba.posseg as pseg
words = pseg.cut(text)
for word, flag in words:
    print('单词：{0}, 词性：{1}'.format(word, flag))

单词：小张, 词性：n
单词：毕业, 词性：n
单词：于, 词性：p
单词：深圳大学, 词性：nt
单词：，, 词性：x
单词：这, 词性：r
单词：座, 词性：q
单词：大学, 词性：n
单词：位于, 词性：v
单词：南山区, 词性：ns


### 关键词提取

Jieba的关键词提取功能，是基于TF-IDF算法的

In [51]:
import jieba.analyse as analyse
text = """欧冠提前一轮出线，近四场比赛取得3胜1平，距离终结联赛对巴萨的不胜纪录也只有一步之遥，
马德里竞技似乎已经完全从惨败威斯特法伦一役的阴霾中走了出来。
球队近来的成绩有所提升，但困扰西蒙尼的战术难题并没有得到解决，
马德里竞技要取得一场的胜利似乎总是要付出比其他球队更多的努力，双线战场18战仅仅打入26球，
场均丢球数却达到了数年来的峰值，联赛中的两大竞争对手均状态不佳，欧冠分组也十分有利，
但马德里竞技依然没能如人们预期的那样脱颖而出，更为尴尬的是，
他们此次已经不能像上赛季那样以引援不力作为借口了。"""
print("  ".join(analyse.extract_tags(text, topK=20, withWeight=False, allowPOS=())))

竞技  马德里  欧冠  联赛  球队  一役  西蒙尼  18  26  场均  丢球数  引援  四场  威斯特法伦  上赛季  那样  取得  巴萨  似乎  出线


# 5. 常用模型

## 5.1 TF/ IDF

### 5.1.1 概念

TF-IDF（term frequency–inverse document frequency）是一种用于资讯检索与资讯探勘的常用加权技术。
   
TF-IDF是一种统计方法，用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度。
   
<b>字词的重要性随着它在文件中出现的次数成正比增加，但同时会随着它在语料库中出现的频率成反比下降。</b>
   
TF-IDF加权的各种形式常被搜寻引擎应用，作为文件与用户查询之间相关程度的度量或评级。
   
除了TF-IDF以外，因特网上的搜寻引擎还会使用基于连结分析的评级方法，以确定文件在搜寻结果中出现的顺序。

### 5.1.2 原理

在一份给定的文件里，<b>词频 (term frequency, TF)</b> 指的是某一个给定的词语在该文件中出现的次数。这个数字通常会被归一化（分子一般小于分母 区别于IDF），以防止它偏向长的文件。（同一个词语在长文件里可能会比短文件有更高的词频，而不管该词语重要与否。）

<b>逆向文件频率 (inverse document frequency, IDF)</b> 是一个词语普遍重要性的度量。某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。

某一特定文件内的高词语频率，以及该词语在整个文件集合中的低文件频率，可以产生出高权重的TF-IDF。因此，TF-IDF倾向于过滤掉常见的词语，保留重要的词语。

<b>TF-IDF的主要思想是：</b>

如果某个词或短语在一篇文章中出现的频率TF高，并且在其他文章中很少出现，则认为此词或者短语具有很好的类别区分能力，适合用来分类。

TF-IDF实际上是：TF * IDF，TF词频(Term Frequency)，IDF反文档频率(Inverse Document Frequency)。

TF表示词条在文档d中出现的频率（另一说：TF词频(Term Frequency)指的是某一个给定的词语在该文件中出现的次数）。

IDF的主要思想是：如果包含词条t的文档越少，也就是n越小，IDF越大，则说明词条t具有很好的类别区分能力。

如果某一类文档C中包含词条t的文档数为m，而其它类包含t的文档总数为k，显然所有包含t的文档数n=m+k，当m大的时候，n也大，按照IDF公式得到的IDF的值会小，就说明该词条t类别区分能力不强。

（另一说：IDF反文档频率(Inverse Document Frequency)是指果包含词条的文档越少，IDF越大，则说明词条具有很好的类别区分能力。）

但是实际上，如果一个词条在一个类的文档中频繁出现，则说明该词条能够很好代表这个类的文本的特征，这样的词条应该给它们赋予较高的权重，并选来作为该类文本的特征词以区别与其它类文档。这就是IDF的不足之处.

最简单的直观公式：

<img src='./image/tf1.png' />

<img src='./image/idf.png' />

<img src='./image/tfidf.png' />

学术一些的公式如下：

<img src='./image/tfidf_math.png' />

维基百科的地址:[tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

示例见本文的：3.3.1

### 5.1.3 与余弦相似度的结合应用

余弦相似度的维基百科定义：[Cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

比如两个句子：

句子A: 我喜欢看电视，不喜欢看电影。

句子B: 我不喜欢看电视，也不喜欢看电影。

1. 经过分词与计算词频：

句子A：我 1，喜欢 2，看 2，电视 1，电影 1，不 1，也 0。

句子B：我 1，喜欢 2，看 2，电视 1，电影 1，不 2，也 1。

2. 写出词频向量

句子A：[1, 2, 2, 1, 1, 1, 0]

句子B：[1, 2, 2, 1, 1, 2, 1]

到这里，问题就变成了如何计算这两个向量的相似程度。

我们可以把它们想象成空间中的两条线段，都是从原点（[0, 0, ...]）出发，指向不同的方向。

两条线段之间形成一个夹角，如果夹角为0度，意味着方向相同、线段重合；如果夹角为90度，意味着形成直角，方向完全不相似；如果夹角为180度，意味着方向正好相反。

因此，我们可以通过夹角的大小，来判断向量的相似程度。夹角越小，就代表越相似。

<img src='./image/cos1.png' />

以二维空间为例，上图的a和b是两个向量，我们要计算它们的夹角θ。余弦定理告诉我们，可以用下面的公式求得：

<img src='./image/cos2.png' />

<img src='./image/cos3.png' />

假定a向量是[x1, y1]，b向量是[x2, y2]，那么可以将余弦定理改写成下面的形式：

<img src='./image/cos4.png' />

<img src='./image/cos5.png' />

数学家已经证明，余弦的这种计算方法对n维向量也成立。假定A和B是两个n维向量，A是 [A1, A2, ..., An] ，B是 [B1, B2, ..., Bn] ，则A与B的夹角θ的余弦等于：

<img src='./image/cos6.png' />

使用这个公式，我们就可以得到，句子A与句子B的夹角的余弦。

<img src='./image/cos7.png' />

余弦值越接近1，就表明夹角越接近0度，也就是两个向量越相似，这就叫"余弦相似性"。

所以，上面的句子A和句子B是很相似的，事实上它们的夹角大约为20.3度。

### 5.1.4 找出相似文章的简易算法

（1）使用TF-IDF算法，找出两篇文章的关键词；

（2）每篇文章各取出若干个关键词（比如20个），合并成一个集合，计算每篇文章对于这个集合中的词的词频（为了避免文章长度的差异，可以使用相对词频）；

（3）生成两篇文章各自的词频向量；

（4）计算两个向量的余弦相似度，值越大就表示越相似。

## 5.2 词向量（Word2Vec）

### 5.2.1 概念

自然语言处理的词频处理方法即TF-IDF，这种方法往往只是可以找出一篇文章中比较关键的词语，即找出一些主题词汇。

但无法给出词汇的语义，比如同义词漂亮和美丽意思差不多应该相近，巴黎之于法国等同于北京之于中国。

对于一句话，如何根据上下文推断出中间的词语是什么，或者由某一个词推测出它的上下文一般是什么词语。

这两种不同的思考方式正好对应两种Word2vec模型，即CBOW模型和Skip-gram模型。

所谓的word vector，就是指将单词向量化，将某个单词用特定的向量来表示。

将单词转化成对应的向量以后，就可以将其应用于各种机器学习的算法中去。


一般来讲，词向量主要有两种形式，分别是稀疏向量和密集向量。

<b>所谓稀疏向量</b>，又称为one-hot representation，就是用一个很长的向量来表示一个词，向量的长度为词典的大小N，向量的分量只有一个1，其他全为0，1的位置对应该词在词典中的索引[1]。

举例来说，如果有一个词典[“面条”,”方便面”,”狮子”]，那么“面条”对应的词向量就是[1,0,0]，“方便面”对应的词向量就是[0,1,0]。这种表示方法不需要繁琐的计算，简单易得，但是缺点也不少，比如长度过长（这会引发维数灾难），以及无法体现出近义词之间的关系，比如“面条”和“方便面”显然有非常紧密的关系，但转化成向量[1,0,0]和[0,1,0]以后，就看不出两者有什么关系了,因为这两个向量相互正交。

当然了，用这种稀疏向量求和来表示文档向量效果还不错，清华的长文本分类工具THUCTC使用的就是此种表示方法

<b>密集向量</b>，又称distributed representation，即分布式表示。最早由Hinton提出，可以克服one-hot representation的上述缺点。

基本思路是通过训练将每个词映射成一个固定长度的短向量，所有这些向量就构成一个词向量空间，每一个向量可视为该空间上的一个点[1]。

此时向量长度可以自由选择，与词典规模无关。这是非常大的优势。

还是用之前的例子[“面条”,”方便面”,”狮子”]，经过训练后，“面条”对应的向量可能是[1,0,1,1,0],而“方便面”对应的可能是[1,0,1,0,0]，而“狮子”对应的可能是[0,1,0,0,1]。

这样“面条”向量乘“方便面”=2，而“面条”向量乘“狮子”=0 。这样就体现出面条与方便面之间的关系更加紧密，而与狮子就没什么关系了。这种表示方式更精准的表现出近义词之间的关系，比之稀疏向量优势很明显。可以说这是深度学习在NLP领域的第一个运用（虽然我觉得并没深到哪里去）

### 5.2.2 Skip-gram和CBOW模型

- Skip-gram：如果是用一个词语作为输入，来预测它周围的上下文，那这个模型叫做『Skip-gram 模型』

<img src='./image/skipgram.jpg' />

可以看成是 单个x->单个y 模型的并联，cost function 是单个 cost function 的累加（取log之后）

- CBOW：如果是拿一个词语的上下文作为输入，来预测这个词语本身，则是 『CBOW 模型』

<img src='./image/cbow.jpg' />

与Skip-gram 的模型并联不同，这里是输入变成了多个单词，所以要对输入处理下（一般是求和然后平均），输出的 cost function不变

### 5.2.3 Skip-gram与CBOW的对比 

CBOW是以周围词作为输入，中心词作为目标的网络，所以假设一篇语料中有V个单词，那么CBOW将会以每一个单词作为中心词进行训练，因此会有V次

Skip-gram是以中心词作为输入，周围词作为目标的网络，那么对于一篇语料来说，每一个词都会作为中心词，每个中心词周围大小选择K个，那么将会进行KV次

直观上来看，CBOW训练次数要比skip-gram少，也即精确率不如skip-gram,但是效率高，速度快。

### 5.2.3 实例演示

我们使用Text8Corpus做演示，这个语料库是英文的，大小不到100M。下载地址为:[Text8Zip](http://mattmahoney.net/dc/text8.zip )

考虑到文件有些大， 就不传到github了，大家自行下载

训练及加载模型：

In [52]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
import os
folder = r'./nlpmodel/text8'
modelfile = os.path.join(folder, 'text8.w2v.model')
if os.path.exists(modelfile):
    # 将硬盘中的模型载入
    model = KeyedVectors.load(modelfile)
else:
    # 使用一个很小的英文语料
    # 下载地址 http://mattmahoney.net/dc/text8.zip 
    sentences = word2vec.Text8Corpus(os.path.join(folder, 'text8'))
    # Gensim的word2vec的训练模式由参数sg决定，0: CBOW，1: skip-gram，默认为CBOW
    model = word2vec.Word2Vec(sentences, size=200, window=5, min_count=5)
    # 设置向量为200维，窗口大小为5，忽略掉词频低于5的词
    # 经过一段时间的等待，就训练完成了。
    # 将训练好的模型保存到硬盘，文件名随意
    model.save(os.path.join(folder, 'text8.w2v.model'))

这个例子是非常经典的，根据positive: worman, king, negative: man，推断出最相近的词是queen的例子

其意义是计算一个词d（或者词表），使得该词的向量v(d)与v(a="woman")-v(c="man")+v(b="king")最近

In [53]:
print(model.most_similar(positive=["woman","king"],negative=["man"],topn=10))

d:\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
d:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.6738182306289673), ('throne', 0.575992226600647), ('empress', 0.5739238262176514), ('princess', 0.5734347105026245), ('elizabeth', 0.5587899684906006), ('daughter', 0.555698037147522), ('jadwiga', 0.5538528561592102), ('prince', 0.5492069721221924), ('isabella', 0.545307457447052), ('consort', 0.541345477104187)]


In [89]:
# 计算两个词的相似度
print(model.similarity('mobile', 'phone'))

d:\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
d:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6278037


In [55]:
# 寻找和某个词最相似的词（会输出词和相似度打分，本以为这个如果自己实现的话会很复杂，竟然在包里就提供了相关方法）
print(model.most_similar('good', topn=20))

d:\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
d:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bad', 0.7257406711578369), ('poor', 0.5354788303375244), ('safe', 0.5291586518287659), ('quick', 0.5234116911888123), ('luck', 0.5217357873916626), ('reasonable', 0.5067963004112244), ('simple', 0.49832555651664734), ('really', 0.492895245552063), ('you', 0.48568713665008545), ('happy', 0.4837035536766052), ('silly', 0.4836675524711609), ('fun', 0.4832077622413635), ('pleasant', 0.4797707796096802), ('my', 0.4790000915527344), ('fast', 0.4784563183784485), ('easy', 0.4768497943878174), ('helpful', 0.47399526834487915), ('practical', 0.4722078740596771), ('your', 0.4720571041107178), ('little', 0.4691823422908783)]


In [56]:
# 识别不合群的词
print(model.doesnt_match('breakfast cereal dinner lunch'.split()))

d:\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  
d:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


cereal


In [57]:
# 获得词典中的词
for index, key in enumerate(model.wv.vocab.keys()):
    print(key)
    if index > 100:
        break
print(len(model.wv.vocab.keys()))

anarchism
originated
as
a
term
of
abuse
first
used
against
early
working
class
radicals
including
the
diggers
english
revolution
and
sans
culottes
french
whilst
is
still
in
pejorative
way
to
describe
any
act
that
violent
means
destroy
organization
society
it
has
also
been
taken
up
positive
label
by
self
defined
anarchists
word
derived
from
greek
without
archons
ruler
chief
king
political
philosophy
belief
rulers
are
unnecessary
should
be
abolished
although
there
differing
interpretations
what
this
refers
related
social
movements
advocate
elimination
authoritarian
institutions
particularly
state
anarchy
most
use
does
not
imply
chaos
nihilism
or
anomie
but
rather
harmonious
anti
place
regarded
structures
71290


## 5.3 文档向量 （Doc2Vec）

### 5.3.1 Doc2Vec原理

Doc2Vec 或者叫做 paragraph2vec, sentence embeddings，是一种非监督式算法，可以获得sentences/paragraphs/documents 的向量表达，是 word2vec 的拓展，Doc2Vec的目的是获得文档的一个固定长度的向量表达, 即向量索引是以Doc为单位，而不是以Word为单位。

学出来的向量可以通过计算距离来找 sentences/paragraphs/documents 之间的相似性， 或者进一步可以给文档打标签。

例如首先是找到一个向量可以代表文档的意思，然后可以将向量投入到监督式机器学习算法中得到文档的标签， 例如在情感分析sentiment analysis 任务中，标签可以是 “negative”, “neutral”,”positive”

Doc2Vec也有两种方式来实现：

<b>DBOW (distributed bag of words)</b>

<img src='./image/DBOW.png' />

<b>DM(distributed memory)</b>

<img src='./image/DM.png' />

实战示例

假定有表示文档分类的样本集，如：

In [58]:
import pandas as pd
from IPython.display import display, HTML

doc2vecmodelrawtextpath = './nlpmodel/vacategorymodelrawtext.csv'
rawdata = pd.read_csv(doc2vecmodelrawtextpath, encoding='utf-16', sep='\t')
display(rawdata)

,category,sentence
0,1,"2060 Retirement Fund, Vanguard Variable Insura..."
1,1,"Effective May 1, 2018, the following funds wil..."
2,1,"Effective on or after May 1, 2018, the followi..."
3,1,"Effective on or after May 1, 2018, the followi..."
4,1,The following Investment Options will be avail...
5,1,The following portfolio has been added as an a...
6,1,This fund is available beginning 06/11/2018)
7,1,This fund will be available on or about May 21...
8,1,This fund will be available on or around May 2...
9,3,"Effective August 13, 2018, the Investment Divi..."


下面来看一下，如何将这个样本集训练为Doc2Vec

准备工作：
- 将数据集进行文本清洗
- 将数据集进行Tag处理
- 需要注意的是：Tag由下划线分隔，下划线左边为样本的索引，下划线右边为样本的category，如：1: Added, 2: Closed, 3: Reopen, 4: Merged, 5: Liquidation
- <b>如果Tag按照这种方式进行处理，即达到利用无监督的方式，实现有监督的目的</b>

In [59]:
def getdatasetfordoc2vector(file):
    print('Get doc list begin')
    rawdata = pd.read_csv(file, encoding='utf-16', sep='\t')
    rawdoclist = rawdata['sentence']
    rawdata['cleantext'] = rawdata['sentence'].apply(lambda x: cleardatafordoc2vector(x))
    rawdata['category'] = rawdata['category'].apply(int)
    x_train = []
    for index, row in rawdata.iterrows():
        word_list = rawdata.loc[index, 'cleantext'].lower().strip().split()
        if len(word_list) == 0:
            word_list = ['only', 'for', 'test']
        tagtype = rawdata.loc[index, 'category']
        if not tagtype:
            tagtype = 0
        document = TaggedDocument(word_list, tags=['{0}_{1}'.format(index, tagtype)])
        x_train.append(document)
    print('Get doc list end')
    return x_train, rawdoclist

训练数据集的主方法体：

- 清洗样本
- 对样本进行训练，生成Doc2Vec模型

In [60]:
def traindoc2vec(rawtextfile,
                 modelfolder,
                 modelfilename,
                 vector_size=200,
                 epoch_num=10,
                 needregenerate=True):
    x_train, rawdoclist = getdatasetfordoc2vector(rawtextfile)
    model_dm = doc2vectortrain(modelfolder,
                               modelfilename,
                               x_train,
                               vector_size=vector_size,
                               epoch_num=epoch_num,
                               needregenerate=needregenerate)
    return x_train, rawdoclist, model_dm

训练样本为Doc2Vec模型的方法体：

- 注意默认的训练次数为30
- 特征向量维度为200

In [61]:
def doc2vectortrain(folder, filename, x_train, vector_size=200, epoch_num=30, needregenerate=False):
    print('Train Doc2Vector begin')
    modelfilepath = os.path.join(folder, filename)
    if not os.path.exists(folder):
        os.makedirs(folder)
    if needregenerate or not os.path.exists(modelfilepath):
        model_dm = Doc2Vec(x_train, min_count=1, window=3, vector_size=vector_size, sample=1e-3, negative=5, workers=2)
        model_dm.train(x_train, total_examples=model_dm.corpus_count, epochs=epoch_num)
        model_dm.save(modelfilepath)
    else:
        model_dm = Doc2Vec.load(modelfilepath)
    print('Train Doc2Vector end')
    return model_dm

训练Doc2Vec

In [62]:
rawtextfile = './nlpmodel/vacategorymodelrawtext.csv'
modelfolder = './nlpmodel'
modelfilename = 'vadoccategorydoc2vec.model'
x_train, rawdoclist, model_dm = traindoc2vec(rawtextfile,
                                             modelfolder,
                                             modelfilename,
                                             vector_size=100,
                                             epoch_num=5000,
                                             needregenerate=False)

Get doc list begin
Get doc list end
Train Doc2Vector begin
Train Doc2Vector end


初始化样本集以及Doc2Vec model

In [63]:
doc2vecmodel = None
alltext = None
doc2vecmodelrawtextpath = './nlpmodel/vacategorymodelrawtext.csv'
doc2vecmodelpath = './nlpmodel/vadoccategorydoc2vec.model'

In [64]:
def initialdoc2vecmodel(alltextpath=doc2vecmodelrawtextpath, doc2vecmodelpath=doc2vecmodelpath):
    print("load text begin")
    global alltext
    if alltext is None:
        rawdata = pd.read_csv(alltextpath, encoding='utf-16', sep='\t')
        alltext = rawdata['sentence']
    print("load text end")

    print("load doc2vec model begin")
    global doc2vecmodel
    if doc2vecmodel is None:
        doc2vecmodel = Doc2Vec.load(doc2vecmodelpath)
    print("load doc2vec model end")

根据Model以及样本数据，获得inver_vector列表 (Infer a vector for given post-bulk training document(为给定的批量后培训文档推断一个向量))

In [65]:
def getdoc2vec_inferedvectorlist(doc2vecmodelpath, x_train):
    infered_vectors_list = []
    print("load doc2vec model begin")
    model_dm = Doc2Vec.load(doc2vecmodelpath)
    print("load doc2vec model end")

    print("load train vectors begin")
    for text, label in x_train:
        vector = model_dm.infer_vector(text)
        infered_vectors_list.append(vector)
    print("load train vectors end")
    return infered_vectors_list

方法体：根据给定的sentence以及Doc2Vec model，获得相似度最高的前10个句子

In [66]:
def getmostsimilaritybydoc2vec(sentence):
    global alltext
    global doc2vecmodel
    if alltext is None or doc2vecmodel is None:
        initialdoc2vecmodel()
    test_cut = cleardatafordoc2vector(sentence).split()
    inferred_vector = doc2vecmodel.infer_vector(test_cut)
    simsbow = doc2vecmodel.docvecs.most_similar([inferred_vector], topn=10)
    return getcontent(sentence, simsbow, alltext)

In [67]:
def getcontent(rawsentence, simsbow, doclist):
    similarresult = {'rawsentence': rawsentence, 'similarlist': []}
    for i in simsbow:
        similardict = {}
        similar = doclist[int(i[0].split('_')[0])]
        similardict['paraid'] = i[0]
        similardict['similarity'] = i[1]
        similardict['paracontent'] = similar
        similarresult['similarlist'].append(similardict)
    return similarresult

In [68]:
initialdoc2vecmodel()

load text begin
load text end
load doc2vec model begin
load doc2vec model end


示例：根据给定的sentence以及Doc2Vec model，获得相似度最高的前10个句子

In [69]:
# sentence = r'Effective May 1, 2018, the following funds will be available as new investment options under your Policy.'
# sentence = r'Fund variable investment option (the "Investment Option") will be liquidated.'
sentence = r'CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018.'
result = getmostsimilaritybydoc2vec(sentence)
print('###############################')
print('raw sentence is: ')
print(result['rawsentence'])
print('###############################')
print()
for similar in result['similarlist']:
    print('paragraph id: {0}, similarity: {1}'.format(
        similar['paraid'],
        similar['similarity']))
    print('paragraph is:')
    print(similar['paracontent'])
    print('###############################')

d:\python36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


###############################
raw sentence is: 
CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018.
###############################

paragraph id: 32_5, similarity: 0.9833414554595947
paragraph is:
— — — — — — *CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018.
###############################
paragraph id: 34_5, similarity: 0.982774555683136
paragraph is:
*CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018.
###############################
paragraph id: 33_5, similarity: 0.9814426302909851
paragraph is:
3,003 2,601 2,169 2,225 1,859 759 232 55 — — *CTIVP SM – Eaton Vance Floating Rate Income Fund (Class 2) liquidated on April 27, 2018.
###############################
paragraph id: 35_5, similarity: 0.7010773420333862
paragraph is:
Accumulation unit value at beginning of period $1.08 $1.00 $1.03 $1.05 $1.03 $1.00 Accumulation unit value at end of period $1.09 $1.08 $1.00 $1.0